In [1]:
from scipy import stats
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
(1- stats.t.cdf(11.58, df=30))/2

3.380073998471289e-13

In [3]:
stats.t.sf(11.58, df = 30)/2

3.379765819515072e-13

In [2]:
df_reviews = pd.read_csv('data/movie_reviews.csv')

model_reviews = smf.ols('rating ~ word_count', data = df_reviews).fit()

model_reviews.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     134.1
Date:                Mon, 04 Sep 2023   Prob (F-statistic):           3.47e-29
Time:                        12:36:02   Log-Likelihood:                -891.54
No. Observations:                1000   AIC:                             1787.
Df Residuals:                     998   BIC:                             1797.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.4916      0.042     82.431      0.000       3.409       3.575
word_count    -0.0427      0.004    -11.580      0.000      -0.050      -0.035
==============================================================================
Omnibus:                        5.980   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.050   Jarque-Bera (JB):                7.341
Skew:                          -0.061   Prob(JB):                       0.0255
Kurtosis:                       3.402   Cond. No.                         26.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
model_reviews_extra = smf.ols(
    formula = 'rating ~ word_count + age + review_year + release_year + length_minutes + children_in_home + total_reviews',
    data = df_reviews
).fit()

model_reviews_extra.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.321
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     67.02
Date:                Mon, 04 Sep 2023   Prob (F-statistic):           3.73e-79
Time:                        15:08:14   Log-Likelihood:                -760.95
No. Observations:                1000   AIC:                             1538.
Df Residuals:                     992   BIC:                             1577.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -45.5688      7.463     -6.106      0.000     -60.215     -30.923
word_count          -0.0303      0.003     -9.102      0.000      -0.037      -0.024
age                 -0.0017      0.001     -1.825      0.068      -0.004       0.000
review_year          0.0099      0.003      3.055      0.002       0.004       0.016
release_year         0.0133      0.002      7.434      0.000       0.010       0.017
length_minutes       0.0167      0.002     10.897      0.000       0.014       0.020
children_in_home     0.1028      0.025      4.051      0.000       0.053       0.153
total_reviews     7.616e-05   6.16e-06     12.362      0.000    6.41e-05    8.83e-05
==============================================================================
Omnibus:                        6.766   Durbin-Watson:                   2.054
Prob(Omnibus):                  0.034   Jarque-Bera (JB):                6.703
Skew:                          -0.178   Prob(JB):                       0.0350
Kurtosis:                       3.185   Cond. No.                     2.82e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.82e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
from inspect import getmembers, isfunction

getmembers(model_reviews_extra)

[('HC0_se',
  Intercept           7.624861
  word_count          0.003827
  age                 0.000943
  review_year         0.003295
  release_year        0.001729
  length_minutes      0.001547
  children_in_home    0.025117
  total_reviews       0.000006
  dtype: float64),
 ('HC1_se',
  Intercept           7.655545
  word_count          0.003842
  age                 0.000947
  review_year         0.003308
  release_year        0.001736
  length_minutes      0.001554
  children_in_home    0.025218
  total_reviews       0.000006
  dtype: float64),
 ('HC2_se',
  Intercept           7.662837
  word_count          0.003853
  age                 0.000947
  review_year         0.003310
  release_year        0.001738
  length_minutes      0.001555
  children_in_home    0.025282
  total_reviews       0.000006
  dtype: float64),
 ('HC3_se',
  Intercept           7.701074
  word_count          0.003879
  age                 0.000951
  review_year         0.003326
  release_year        0.001

In [40]:
model_reviews_extra.params.ind

AttributeError: 'OLSResults' object has no attribute 'names'

In [3]:
# we can also use the shap package for this
import shap
from sklearn.linear_model import LinearRegression

# my god
# x_wc = model_reviews.model.exog
# x12 = np.array([12.0])

X = df_reviews[model_reviews_extra.model.exog_names[1:]]
# model_reviews = LinearRegression().fit(model_reviews_extra.model.exog, df_reviews['rating'])
model_reviews = LinearRegression().fit(X, df_reviews['rating'])

explainer = shap.explainers.Linear(model_reviews, X, algorithm = 'linear')
shap_values = explainer(X.assign(word_count = 12))

shap_values[:, 0].mean(0)

# model_reviews.coef_

# shap_value_package = explainer(df_reviews.assign(word_count = 12))

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


NameError: name 'model_reviews_extra' is not defined

In [4]:
feature_to_analyze = "word_count"
sample_index = 0
fixed_value = 12
X = df_reviews[feature_to_analyze].values.reshape(-1, 1)
X_samp_idx = df_reviews.iloc[-3][feature_to_analyze]
# model_reviews = LinearRegression().fit(model_reviews_extra.model.exog, df_reviews['rating'])
model_reviews = LinearRegression().fit(X, df_reviews['rating'])
fixed_value_df = np.array(df_reviews.assign(**{feature_to_analyze: fixed_value})[feature_to_analyze]).reshape(-1, 1)
avg_pred_fixed_value_samp_idx = model_reviews.predict(fixed_value_df).mean()

avg_pred_fixed_value_samp_idx

2.9794358383461677

Compare to average prediction.

In [15]:
avg_pred = model_reviews.predict(X).mean()
avg_pred

3.0513999999999997

In [29]:
model_reviews.predict(X[np.where(X != 12)].reshape(-1, 1)).mean()

3.053855860738962

In [50]:
type(np.array(X_samp_idx))

numpy.ndarray

In [6]:
explainer_linear = shap.Explainer(
    model_reviews.predict, 
    shap.utils.sample(X, 10000),
    algorithm='linear'
)
shap_values_linear = explainer_linear(X)
shap_values_linear[-3]

InvalidModelError: An unknown model type was passed: <class 'method'>

Compare result to the shap value at an observation that is at the first value of the feature to analyze.

In [ ]:
pd.DataFrame({'shap_calc': [avg_pred_fixed_value_samp_idx - avg_pred], 
              'shap_value': shap_values_linear.values[sample_index, 0]})

,shap_calc,shap_value
0,-0.071964,-0.075976


Exact code from demo.py

In [26]:
from sklearn.linear_model import LinearRegression


X = np.array(df_reviews['word_count']).reshape(-1, 1)
y = df_reviews['rating']
model_reviews = LinearRegression().fit(X, y)


sample_index = -3

X_sample = shap.utils.sample(X, 100)  # 100 instances for use as the background distribution

# # compute the SHAP values for the linear model
explainer_linear = shap.Explainer(model_reviews.predict, X_sample)
shap_values_linear = explainer_linear(X)

In [27]:
avg_pred = model_reviews.predict(X_sample).mean()
avg_pred

3.055412236770735

In [29]:
shap_values_linear[sample_index]

.values =
array([-0.0759764])

.base_values =
3.0554122367707364

.data =
array([12])

NameError: name 'model_reviews_extra' is not defined